In [8]:
import requests
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
from delta import DeltaTable

spark = (SparkSession.builder
             .appName('lab') # Name the app
             .config("hive.metastore.uris", "thrift://metastore:9083") # Set external Hive Metastore
             .config("hive.metastore.schema.verification", "false") # Prevent some errors
             .config("spark.sql.repl.eagerEval.enabled", True)
             .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
             .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
             .enableHiveSupport()
             .getOrCreate())

In [2]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
| business|
|  default|
|    stage|
+---------+



In [40]:
spark.sql("show tables from business").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| business|  posicao|      false|
+---------+---------+-----------+



In [34]:
df_previsao_t.filter("cp = '100014344'")

cp,c,cl,lt0,lt1,qv,sl,a,p,px,py,ta,np,px,py
100014344,2123-10,33460,LIBERDADE,VL. MEDEIROS,1,2,true,21322,-46.616203,-23.542707,2024-09-23T23:15:28Z,,-46.616608,-23.535778
100014344,271F-10,33451,METRÔ BELÉM,CENTER NORTE,1,2,true,21728,-46.607777,-23.539167,2024-09-23T23:15:23Z,,-46.616608,-23.535778
100014344,701A-10,689,METRÔ VL. MADALENA,PQ. EDU CHAVES,2,1,true,21386,-46.594788,-23.524677,2024-09-23T23:15:34Z,,-46.616608,-23.535778
100014344,701A-10,689,METRÔ VL. MADALENA,PQ. EDU CHAVES,2,1,true,21871,-46.576435,-23.488453,2024-09-23T23:15:15Z,,-46.616608,-23.535778


In [47]:
spark.table("business.previsao").show()

+-----+-------+-------+-----+--------------------+--------------------+---+---+----+-----+----------+----------+--------------------+--------------------+----------+----------+
|   hr|     cp|      c|   cl|                 lt0|                 lt1| qv| sl|vs_a| vs_p|     vs_px|     vs_py|               vs_ta|                  np|        px|        py|
+-----+-------+-------+-----+--------------------+--------------------+---+---+----+-----+----------+----------+--------------------+--------------------+----------+----------+
|20:03| 404598|407Y-10| 2206|  METRÔ VL. PRUDENTE|      PQ. SAVOY CITY|  1|  1|true|48033|-46.509346| -23.57051|2024-09-23T23:03:16Z|                    | -46.51706|-23.576618|
|20:03| 600539|875P-10|34109|      METRÔ ANA ROSA|   METRÔ BARRA FUNDA|  2|  2|true|82754| -46.66222| -23.55635|2024-09-23T23:03:12Z|                    | -46.66213| -23.53247|
|20:03| 614547|2014-10|  900|   METRÔ BARRA FUNDA|       JD. PERY ALTO|  5|  1|true|25015| -46.66427|-23.484724|202